# HowTo get SDSS photometrical data

В этом туториале показано, как можно извлечь фотометрические данные Слоановского обзора неба (Sloan Digital Sky Survey - SDSS) с помощью сервиса [`SkyServer`](http://skyserver.sdss.org/dr15/en/home.aspx).

## Импорт таблицы скоплений галактик

Мы будем работать с данными по скоплениям галактик, которые построены по SDSS в работу [Saulder et al. 2016](https://ui.adsabs.harvard.edu/abs/2016A%26A...596A..14S/abstract). Эту таблицу удобно напрямую импортировать из
астрономического "каталога каталогов" [`Vizier`](http://vizier.u-strasbg.fr/viz-bin/VizieR-2). Можно руками импортировать с [веба](http://vizier.u-strasbg.fr/viz-bin/VizieR-3?-source=J/A%2bA/596/A14/grlist_s&-out.max=50&-out.form=HTML%20Table&-out.add=_r&-out.add=_RAJ,_DEJ&-sort=_r&-oc.form=sexa), но мы для удобства воспользуемся аффилированой с `astropy` библиотекой `astroquery`, которая еще пригодится ниже. Импортируем ее и устанавливаем `ROW_LIMIT = -1`, чтобы `Vizier` мог отдать полную таблицу, по умолчанию он отдает первые 50 записей. В исходной таблице `grlist` приводятся 229893 строк, которые включают не только скопления и группы галактик, но и отдельные галактики, которые не были присовокуплены ни к одной из структур. Для дальнейшей работы нам понадобятся лишь хорошие скопления/группы, к которы были приписаны десятки скоплений. В таблице есть колонка `Ntot`, которая показывает число галактик в скоплении/группе. При запросе в `Vizier` сразу попросим его выдавать скопления с `Ntot > 70`, таких будет чуть больше 100. На последующих этапах проекта можно будет изменить этот порог.


In [53]:
from astroquery.vizier import Vizier
Vizier.ROW_LIMIT = -1
grplist = Vizier.query_constraints(catalog="J/A+A/596/A14/grlist_s", Ntot=">70")[0]
grplist

iGrID,RAJ2000,DEJ2000,z,logLtot,logLobs,logMtot,logMstar,NMstar,logMdyn,sigma,Rad,angRad,DL,Ntot,Dist
,deg,deg,,[Lsun],[Lsun],[Msun],[Msun],,[Msun],km / s,kpc,deg,Mpc,,
int32,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64,float64,int32,bytes1
211,195.72035217,-2.52054167,0.0840534,12.47245,12.32273,14.99262,12.81219,91,12.4724,647.4734,1015.1303,0.184561,370.3425,91,*
1195,170.83279419,1.02137434,0.0749820,12.27665,12.15536,14.78539,12.55322,73,12.3085,535.2502,1018.4297,0.205672,327.8531,73,*
4766,164.55435181,1.60687900,0.0405723,12.09625,12.05222,14.52421,12.39123,75,11.8741,370.5783,781.5141,0.281270,172.3759,81,*
5742,181.11277771,1.89597118,0.0212014,11.80343,11.79065,14.41131,12.30175,80,11.8652,487.7089,442.0396,0.297936,88.6502,82,*
7892,243.49198914,49.18959808,0.0576766,12.26128,12.18258,14.70631,12.62276,84,12.1443,414.4820,1163.6238,0.300066,248.5541,87,*
10476,184.42135620,3.65584564,0.0786997,12.59475,12.46241,15.15141,12.96925,130,12.7001,864.9491,960.7928,0.185565,345.1888,134,*
11499,214.39805603,2.05321932,0.0550035,12.36745,12.29465,14.84609,12.69558,116,12.2693,482.4626,1145.3145,0.308821,236.5068,121,*
12324,168.46244812,2.48143363,0.0772917,12.53759,12.40959,15.08015,12.73734,116,12.6500,634.7978,1589.5846,0.312153,338.6106,125,*


## Из влечение данных с помощью `SDSS SkyServer`


В фотометрчиеской части обзора SDSS содержится информация об очень большом количестве объектов на небе ~0.5 млрд. Поэтому извлекать полный dataset и работать с ним не имеет смысла для нашей задачи. Нам всего лишь нужно извлечь некоторые данные для областей неба около исследуемых скоплений/групп. Причем, как видно из таблицы `grplist`, разные скопления/группы имеют разный угловой размер на небе (колонка `AngRad`).

Есть несколько способов извлечь фотометрические данные доступные в обзоре SDSS (Table Access Protocol (TAP), [CasJobs](https://skyserver.sdss.org/casjobs/)). Все они заключаются в работе с составлением SQL (или ADQL - SQL для работы с TAP) запросов. Но оказывается достаточно сложно составиь такой запрос, где будут извлекаться данные для каждого скопления с разным угловым размером. Поэтому более простой способ оказался вопспользовать сервисом `SkyServer` и выполнить индивидуаьные запросы для каждого скопления. Пример запроса "чистых" (без больших инструментальных проблем) фотометрических данных приведен [тут](http://skyserver.sdss.org/dr15/en/help/docs/realquery.aspx#cleanGals). Потренироваться в написании различных SQL запросов можно в [SkyServer SQL Form](http://skyserver.sdss.org/dr15/en/tools/search/sql.aspx).

Мы же напишем небольшой скрипт для выполнения запросов и сохранения результатов на диск в `csv` файлы для каждого скопления.

### Замечания
- Некоторые скопления очень большие и чтобы запрос переварился нужно увеличить timeout до 10 минут, которого хватает для успешного завершения большинства запросов.
- Иногда запрос валится и требуется повторно запустить эту ячейку. Запросы кэшируются, поэтому при повторном выполнение время исполнения запроса оказывается не репрезентативным.
- Чтобы заведомо покрыть все скопления, данные извлекаются с области в `factor = 1.5` раз больше, чем размер скопления `angRad`.

In [97]:
import time
from astroquery.sdss import SDSS

# Some clusters are huge and long time is required to proccessed query,
# so let's increase default timeout from 1 minute to 5 min
SDSS.TIMEOUT = 600

factor = 1.5

for idx, grp in enumerate(grplist):
    query = (
"""
SELECT
    g.objid, g.ra, g.dec, g. dered_g, g. dered_r, err_g, err_r, petroRad_r, petroRadErr_r
FROM
    Galaxy AS g,
    dbo.fGetNearbyObjEq({},{},{}) AS n 
WHERE
    g.objID = n.objID
    AND ((g.flags_r & 0x10000000) != 0)
        -- detected in BINNED1
    AND ((g.flags_r & 0x8100000c00a0) = 0)
        -- not NOPROFILE, PEAKCENTER, NOTCHECKED, PSF_FLUX_INTERP, SATURATED,
        -- or BAD_COUNTS_ERROR.
        -- if you want to accept objects with interpolation problems for PSF mags,
        -- change this to: AND ((flags_r & 0x800a0) = 0)
    AND (((g.flags_r & 0x400000000000) = 0) or (g.psfmagerr_r <= 0.2))
        -- not DEBLEND_NOPEAK or small PSF error
        -- (substitute psfmagerr in other band as appropriate)
    AND (((g.flags_r & 0x100000000000) = 0) or (g.flags_r & 0x1000) = 0)
        -- not INTERP_CENTER or not COSMIC_RAY - omit this AND clause if you want to
        -- accept objects with interpolation problems for PSF mags.
""".format(grp['RAJ2000'], grp['DEJ2000'], grp['angRad']*60.0*factor)
    )
    
    msg = "[{:d}/{:d}] Working on iGrID = {:<8d} (Ntot = {:<3d} Rad = {:<0.1f} arcmin) at RA={}; DEC={}"
    print(msg.format(idx+1, len(grplist), grp['iGrID'], grp['Ntot'],
                     grp['RAJ2000'], grp['DEJ2000'], grp['angRad']))
    
    # make query
    tstart = time.process_time()
    tbl = SDSS.query_sql(query, data_release=15)
    
    # save table to csv file 
    ofile = "phot_sdss_grps/phot_gals_irgp_{}.csv".format(grp['iGrID'])
    tbl.write(ofile, format="ascii.csv", overwrite=True)
    print("    Extracted {} galaxies in {:.1f} sec, saved to {}".format(len(tbl), time.process_time() - tstart, ofile)) 


[1/107] Working on iGrID = 211      (Ntot = 91  Rad = 195.7 arcmin) at RA=-2.52054167; DEC=0.184561
    Extracted 2478 galaxies in 0.1 sec, saved to phot_sdss_grps/phot_gals_irgp_211.csv
[2/107] Working on iGrID = 1195     (Ntot = 73  Rad = 170.8 arcmin) at RA=1.02137434; DEC=0.205672
    Extracted 3549 galaxies in 0.1 sec, saved to phot_sdss_grps/phot_gals_irgp_1195.csv
[3/107] Working on iGrID = 4766     (Ntot = 81  Rad = 164.6 arcmin) at RA=1.606879; DEC=0.28127
    Extracted 4624 galaxies in 0.1 sec, saved to phot_sdss_grps/phot_gals_irgp_4766.csv
[4/107] Working on iGrID = 5742     (Ntot = 82  Rad = 181.1 arcmin) at RA=1.89597118; DEC=0.297936
    Extracted 6317 galaxies in 0.2 sec, saved to phot_sdss_grps/phot_gals_irgp_5742.csv
[5/107] Working on iGrID = 7892     (Ntot = 87  Rad = 243.5 arcmin) at RA=49.18959808; DEC=0.300066
    Extracted 7719 galaxies in 0.3 sec, saved to phot_sdss_grps/phot_gals_irgp_7892.csv
[6/107] Working on iGrID = 10476    (Ntot = 134 Rad = 184.4 arcmin)

    Extracted 4987 galaxies in 0.2 sec, saved to phot_sdss_grps/phot_gals_irgp_58282.csv
[45/107] Working on iGrID = 58643    (Ntot = 72  Rad = 207.9 arcmin) at RA=46.36671448; DEC=0.19702
    Extracted 3377 galaxies in 0.2 sec, saved to phot_sdss_grps/phot_gals_irgp_58643.csv
[46/107] Working on iGrID = 62024    (Ntot = 93  Rad = 127.1 arcmin) at RA=30.43132019; DEC=0.146149
    Extracted 1551 galaxies in 0.1 sec, saved to phot_sdss_grps/phot_gals_irgp_62024.csv
[47/107] Working on iGrID = 64669    (Ntot = 87  Rad = 175.7 arcmin) at RA=8.16227913; DEC=0.35827
    Extracted 11467 galaxies in 0.5 sec, saved to phot_sdss_grps/phot_gals_irgp_64669.csv
[48/107] Working on iGrID = 65627    (Ntot = 110 Rad = 186.9 arcmin) at RA=8.82488155; DEC=0.180053
    Extracted 2847 galaxies in 0.1 sec, saved to phot_sdss_grps/phot_gals_irgp_65627.csv
[49/107] Working on iGrID = 67012    (Ntot = 74  Rad = 192.2 arcmin) at RA=55.06230927; DEC=0.255378
    Extracted 5563 galaxies in 0.2 sec, saved to phot

    Extracted 8360 galaxies in 0.4 sec, saved to phot_sdss_grps/phot_gals_irgp_149469.csv
[89/107] Working on iGrID = 149473   (Ntot = 896 Rad = 240.6 arcmin) at RA=16.34599113; DEC=0.942822
    Extracted 82263 galaxies in 3.2 sec, saved to phot_sdss_grps/phot_gals_irgp_149473.csv
[90/107] Working on iGrID = 150207   (Ntot = 143 Rad = 229.9 arcmin) at RA=20.89632034; DEC=0.618485
    Extracted 33293 galaxies in 1.3 sec, saved to phot_sdss_grps/phot_gals_irgp_150207.csv
[91/107] Working on iGrID = 152483   (Ntot = 140 Rad = 234.9 arcmin) at RA=21.78271484; DEC=0.285113
    Extracted 6898 galaxies in 0.3 sec, saved to phot_sdss_grps/phot_gals_irgp_152483.csv
[92/107] Working on iGrID = 153084   (Ntot = 765 Rad = 195.0 arcmin) at RA=27.97694016; DEC=0.717534
    Extracted 44686 galaxies in 2.0 sec, saved to phot_sdss_grps/phot_gals_irgp_153084.csv
[93/107] Working on iGrID = 153748   (Ntot = 75  Rad = 219.4 arcmin) at RA=24.85753822; DEC=0.280763
    Extracted 6569 galaxies in 0.3 sec, sa

Полный размер сохраненных csv файлов составляет чуть менее 200mb:
```
ik52@kaluga # data/phot_sdss_grps [master ✗] [12:53:43]
➜ du -h
189M
```

### Что имеем в извлеченных данных?

В выполненных SQL запросах мы извлекали данные о галактиках (используются вьюха `Galaxy`) и попросили вернуть сервер следующие параметры:
- `objid` - стандартный ID SDSS объекта
- `ra`, `dec` - экваториальные небесные координаты
- `dered_g`, `dered_r`, `err_g`, `err_r` - это звездные величины в фильтрах `g` и `r` и их ошибки. `dered_` означает, что эти величины были исправлены за поглащение света в Нашей Галактике. Разность `dered_g - dered_r` - это оптический цвет галактики. Если одинаковую галактику помещать на разное красное смещение, цвет галактики будет изменяться из-за космологического расширения, которое на больших масштабах уже будет заметно. Напомню, что звездная величина - это $-2.5\log (F) + Z_{point}$, где $F$ - это полный поток излучения, приходящий от галактики в данном фильтре, $Z_{point}$ - нуль-пункт.
- `petroRad_R` и `petroRadErr_r` - [петросяновский радиус (Petrosian radius)](http://skyserver.sdss.org/dr7/en/help/docs/algorithm.asp?key=mag_petro) и ошибка в фильре `r`. Такой радиус является модельно незасимым и его можно сипользовать как индикатор размера галактики.

In [86]:
tbl

objid,ra,dec,dered_g,dered_r,err_g,err_r,petroRad_r,petroRadErr_r
int64,float64,float64,float64,float64,float64,float64,float64,float64
1237651752929722799,164.314764778059,1.70463320738435,22.66586,22.00591,0.2167703,0.1722437,1.851976,1.67056
1237651752929722798,164.312940162349,1.6867846504343,21.88967,20.95791,0.1952541,0.1234173,3.00205,0.3384922
1237651752929722796,164.311497433452,1.68143215679143,22.31338,21.3545,0.2135836,0.1327558,2.577923,0.6609415
1237651752929722786,164.306495034618,1.69032793840833,21.60943,20.91613,0.09827613,0.07628441,2.059687,0.234718
1237651752929722782,164.305636773052,1.6556597773296,22.56131,21.38212,0.1964237,0.09581868,1.461135,0.142469
1237651752929722780,164.305343728178,1.66006118331451,22.44976,21.86393,0.1609097,0.1296817,1.290057,0.196543
1237654028714901623,164.593078795242,1.48065016988491,21.52532,19.82549,0.1287903,0.0433135,3.301496,0.6890357
1237651752929854191,164.640625102637,1.57444441687142,22.86213,21.97291,0.2437011,0.1607508,1.457518,0.1937261
1237651752929854189,164.638880286984,1.56482279869691,22.99885,21.89099,0.241629,0.1312981,1.379502,0.2325576
